In [1]:
import sqlite3
import requests
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
import os
import requests
import time
from datetime import datetime, timedelta, timezone
import requests
import pandas as pd
import time


In [2]:
conn = sqlite3.connect("Database/news_data.db")

In [11]:
df = pd.read_sql_query("""
                        SELECT 
                            ticker
                        FROM Stocks
                        """, conn)

tickers = df["ticker"].values.tolist()

In [27]:
dfs_list = []
for ticker in tickers:
    query = (f"""
                SELECT a.ticker,
                    a.url,
                    a.title,
                    a.source_domain,
                    a.closest_time_before_published,
                    datetime(a.closest_time_before_published, '+1 hour') as '1 hour after published',
                    CASE
                        WHEN TIME(REPLACE(a.closest_time_before_published, '+00:00', ''), '+1 hour') > '19:50:00'
                        THEN DATE(REPLACE(a.closest_time_before_published, '+00:00', ''), '+1 day') || ' 13:30:00'
                        ELSE DATETIME(REPLACE(a.closest_time_before_published, '+00:00', ''), '+1 hour')
                    END AS future_time,
                    s.open as open_price,
                    f.close AS future_price,
                    a.time_published
                FROM {ticker}_NewsArticles AS a
                LEFT JOIN {ticker}PriceHistory AS s 
                    ON a.closest_time_before_published = s.date
                LEFT JOIN {ticker}PriceHistory AS f 
                    ON REPLACE(f.date, '+00:00', '') = future_time
                """)

    df = pd.read_sql_query(query, conn)
    dfs_list.append(df)

In [51]:
dfs_list[0]['source_domain']

test = pd.DataFrame()

for df in dfs_list:
    if not df.empty:
        grouped = df['source_domain']
        test = pd.concat([test, grouped], ignore_index=True)

In [55]:
len(test.drop_duplicates())

52

In [32]:
final_df = pd.DataFrame()

for df in dfs_list:
    if not df.empty:
        ticker = df["ticker"][0]
        grouped = df.groupby('source_domain').size().reset_index(name=f'{ticker}_frequency')
        final_df = pd.concat([final_df, grouped], ignore_index=True)

"""final_df = final_df.groupby('source_domain').sum().reset_index()
print(final_df.sort_values(by='frequency', ascending=False))"""

"final_df = final_df.groupby('source_domain').sum().reset_index()\nprint(final_df.sort_values(by='frequency', ascending=False))"

In [34]:
final_df["sum_frequency"] = final_df.filter(like='_frequency').sum(axis=1)

In [42]:
grouped_final_df = final_df.groupby('source_domain').sum(numeric_only=True).reset_index()

In [45]:
len(grouped_final_df)

52

In [38]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

In [62]:

grouped_final_df[grouped_final_df["sum_frequency"] > 500].sort_values(by='sum_frequency', ascending=False)

,source_domain,AAPL_frequency,MSFT_frequency,NVDA_frequency,BAC_frequency,GS_frequency,PLD_frequency,O_frequency,SPG_frequency,XOM_frequency,CVX_frequency,SLB_frequency,AMZN_frequency,WMT_frequency,TGT_frequency,KO_frequency,PG_frequency,PEP_frequency,JNJ_frequency,PFE_frequency,NFLX_frequency,DIS_frequency,sum_frequency
23,www.benzinga.com,697.0,406.0,321.0,296.0,503.0,85.0,78.0,134.0,552.0,545.0,531.0,332.0,465.0,405.0,259.0,284.0,412.0,436.0,443.0,384.0,488.0,8056.0
51,www.zacks.com,345.0,263.0,257.0,138.0,116.0,217.0,267.0,397.0,1064.0,1137.0,871.0,340.0,127.0,544.0,305.0,340.0,238.0,336.0,244.0,234.0,145.0,7925.0
33,www.fool.com,360.0,260.0,426.0,417.0,196.0,199.0,855.0,98.0,600.0,855.0,24.0,618.0,274.0,280.0,516.0,302.0,399.0,208.0,285.0,242.0,59.0,7473.0
39,www.investors.com,98.0,36.0,77.0,18.0,22.0,5.0,2.0,18.0,52.0,130.0,158.0,60.0,16.0,42.0,20.0,45.0,12.0,25.0,9.0,26.0,3.0,874.0
27,www.cnbc.com,11.0,55.0,88.0,121.0,65.0,2.0,5.0,36.0,84.0,6.0,34.0,11.0,61.0,9.0,23.0,51.0,41.0,37.0,50.0,33.0,21.0,844.0
37,www.globenewswire.com,0.0,30.0,10.0,39.0,52.0,6.0,0.0,10.0,96.0,10.0,84.0,1.0,24.0,3.0,44.0,40.0,34.0,117.0,140.0,36.0,19.0,795.0
34,www.forbes.com,10.0,53.0,40.0,18.0,20.0,12.0,3.0,32.0,72.0,24.0,14.0,2.0,47.0,11.0,44.0,31.0,36.0,38.0,31.0,176.0,52.0,766.0
45,www.prnewswire.com,2.0,29.0,11.0,16.0,15.0,20.0,35.0,20.0,40.0,20.0,34.0,0.0,26.0,15.0,34.0,61.0,69.0,53.0,59.0,15.0,75.0,649.0


In [64]:
grouped_final_df[grouped_final_df["sum_frequency"] > 500].sort_values(by='sum_frequency', ascending=False)[["source_domain","sum_frequency"]]

,source_domain,sum_frequency
23,www.benzinga.com,8056.0
51,www.zacks.com,7925.0
33,www.fool.com,7473.0
39,www.investors.com,874.0
27,www.cnbc.com,844.0
37,www.globenewswire.com,795.0
34,www.forbes.com,766.0
45,www.prnewswire.com,649.0
